## Preprocesamiento

In [1]:
import pandas as pd
import nltk
import string
import os
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\isaac\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\isaac\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
RATING_COLS = ['user', 'rating', 'ID', 'comment']
GAME_INFO_COLS = ['id', 'primary', 'yearpublished', 'minplayers', 'maxplayers',
                  'playingtime', 'boardgamecategory', 'boardgamemechanic',
                  'boardgamefamily', 'boardgamedesigner', 'boardgamepublisher',
                  'usersrated', 'average', 'Board Game Rank', 'numcomments',
                  'averageweight']

# Rutas de los archivos CSV (ajustar según la ubicación de tus archivos)
RATINGS_CSV = "../data/bgg-26m-reviews.csv"         # Archivo con ratings y comentarios
GAMES_CSV = "../data/games_detailed_info.csv"         # Archivo con información de juegos


## Funciones

In [6]:
def load_ratings(ratings_file=RATINGS_CSV):
    """
    Carga el CSV de ratings utilizando las columnas definidas y renombra 'ID' a 'game_id'.
    """
    if not os.path.exists(ratings_file):
        raise FileNotFoundError(f"El archivo {ratings_file} no se encontró.")
    ratings_df = pd.read_csv(ratings_file, usecols=RATING_COLS, low_memory=False)
    ratings_df.rename(columns={'ID': 'game_id'}, inplace=True)
    return ratings_df

In [7]:
def load_game_info(games_file=GAMES_CSV):
    """
    Carga el CSV de información de juegos utilizando las columnas definidas y renombra 'id' a 'game_id'.
    """
    if not os.path.exists(games_file):
        raise FileNotFoundError(f"El archivo {games_file} no se encontró.")
    games_df = pd.read_csv(games_file, usecols=lambda col: col in GAME_INFO_COLS, low_memory=False)
    if 'id' in games_df.columns:
        games_df.rename(columns={'id': 'game_id'}, inplace=True)
    return games_df

In [ ]:

def clean_data(df, columns_to_check):
    """
    Elimina duplicados y filas con valores nulos en las columnas especificadas.
    """
    df_cleaned = df.drop_duplicates()
    
    df_cleaned = df_cleaned.dropna(subset=columns_to_check)
    return df_cleaned

In [9]:
def optimize_memory_usage(df):
    """
    Convierte las columnas de tipo object a string para optimizar el uso de memoria.
    """
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype("string")
    return df

In [10]:
def create_combined_text(games_df):
    """
    Combina las columnas textuales relevantes del dataframe de juegos para formar
    una única columna 'combined_text' que servirá para el análisis basado en contenido.
    Se utilizan: 'primary', 'boardgamecategory', 'boardgamemechanic',
    'boardgamefamily', 'boardgamedesigner' y 'boardgamepublisher'.
    """
    text_columns = ['primary', 'boardgamecategory', 'boardgamemechanic', 
                    'boardgamefamily', 'boardgamedesigner', 'boardgamepublisher']
    
    # Rellenar con cadena vacía si la columna no existe o tiene valores nulos
    for col in text_columns:
        if col in games_df.columns:
            games_df[col] = games_df[col].fillna("").astype(str)
        else:
            games_df[col] = ""
    
    # Combinar las columnas en una sola cadena
    games_df['combined_text'] = games_df[text_columns].apply(lambda row: " ".join(row.values.astype(str)), axis=1)
    return games_df

In [11]:
def preprocess_text(text, stemmer, stop_words):
    """
    Preprocesa el texto: lo convierte a minúsculas, elimina puntuación,
    tokeniza, elimina stopwords y aplica stemming.
    """
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return " ".join(filtered_tokens)

In [12]:
def apply_text_preprocessing(df, source_column, target_column):
    """
    Aplica el preprocesamiento de texto a la columna 'source_column' y almacena el resultado en 'target_column'.
    """
    stemmer = SnowballStemmer('spanish')
    stop_words = set(stopwords.words('spanish'))
    df[target_column] = df[source_column].apply(lambda x: preprocess_text(x, stemmer, stop_words))
    return df

In [13]:
def aggregate_ratings(ratings_df):
    """
    Agrupa los ratings por 'game_id' calculando el promedio y la cantidad de ratings.
    Además, concatena todos los comentarios en una única cadena por juego.
    """
    aggregated = ratings_df.groupby('game_id').agg(
        average_rating=pd.NamedAgg(column='rating', aggfunc='mean'),
        num_ratings=pd.NamedAgg(column='rating', aggfunc='count'),
        aggregated_comments=pd.NamedAgg(column='comment', aggfunc=lambda x: " ".join(x.astype(str)))
    ).reset_index()
    return aggregated

In [14]:
def merge_datasets(games_df, aggregated_ratings):
    """
    Une el dataframe de juegos con el dataframe de ratings agregados utilizando 'game_id'.
    """
    merged_df = pd.merge(games_df, aggregated_ratings, on='game_id', how='left')
    return merged_df

In [15]:
def main():
    # 1. Cargar datasets
    ratings_df = load_ratings()
    games_df = load_game_info()
    
    # 2. Limpiar datos: eliminar duplicados y registros nulos en columnas esenciales
    ratings_df = clean_data(ratings_df, ['rating', 'user', 'comment'])
    # Añadir eliminación de valores nulos para todos las columnas
    games_df = clean_data(games_df, ['primary'])
    
    # 3. Optimizar uso de memoria
    ratings_df = optimize_memory_usage(ratings_df)
    games_df = optimize_memory_usage(games_df)
    
    # 4. Agregar ratings: calcular promedio, cantidad y concatenar comentarios por juego
    aggregated_ratings = aggregate_ratings(ratings_df)
    
    # 5. Crear columna de texto combinada en games_df a partir de las columnas textuales relevantes
    games_df = create_combined_text(games_df)
    
    # 6. Preprocesar el texto combinado de los juegos
    games_df = apply_text_preprocessing(games_df, 'combined_text', 'processed_game_text')
    
    # 7. Preprocesar los comentarios agregados
    aggregated_ratings = apply_text_preprocessing(aggregated_ratings, 'aggregated_comments', 'processed_comments')
    
    # 8. Unir datasets: información de juegos con ratings agregados
    merged_df = merge_datasets(games_df, aggregated_ratings)
    
    # 9. Crear una columna final que combine el texto preprocesado de juegos y comentarios (si existen)
    merged_df['final_text'] = merged_df['processed_game_text'] + " " + merged_df['processed_comments'].fillna("")
    
    # 10. Guardar el DataFrame final preprocesado en un nuevo archivo CSV
    output_file = "preprocessed_boardgame_data.csv"
    merged_df.to_csv(output_file, index=False)
    print(f"Preprocesamiento completado. Datos guardados en '{output_file}'.")

if __name__ == "__main__":
    main()

KeyboardInterrupt: 